In [ ]:
#未保留checkpoint

In [1]:
!pip install -i http://pypi.douban.com/simple/ --trusted-host=pypi.douban.com/simple pandas
!pip install -i http://pypi.douban.com/simple/ --trusted-host=pypi.douban.com/simple paddlepaddle==1.8.4
!pip install -i http://pypi.douban.com/simple/ --trusted-host=pypi.douban.com/simple pgl==1.2.0 
!pip install -i http://pypi.douban.com/simple/ --trusted-host=pypi.douban.com/simple easydict   

Looking in indexes: http://pypi.douban.com/simple/
Looking in indexes: http://pypi.douban.com/simple/
Looking in indexes: http://pypi.douban.com/simple/


Looking in indexes: http://pypi.douban.com/simple/


In [2]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd

In [3]:
# 训练600epoch acc=0.72
from easydict import EasyDict as edict
config = {
    "model_name": "GCN",
    "num_layers":  3,
    "dropout": 0.5,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "edge_dropout": 0.00,
}
config = edict(config)

In [4]:
from collections import namedtuple

Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])

def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("work/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])
    
    return edges

def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("work/feat.npy")
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})
    
    indegree = graph.indegree()
    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)
    
    df = pd.read_csv("work/train.csv")
    node_index = df["nid"].values
    node_label = df["label"].values
    train_part = int(len(node_index) * 0.8)
    train_index = node_index[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_index[train_part:]
    valid_label = node_label[train_part:]
    test_index = pd.read_csv("work/test.csv")["nid"].values
    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    return dataset


In [5]:
dataset = load()

train_index = dataset.train_index
train_label = np.reshape(dataset.train_label, [-1 , 1])
train_index = np.expand_dims(train_index, -1)

val_index = dataset.valid_index
val_label = np.reshape(dataset.valid_label, [-1, 1])
val_index = np.expand_dims(val_index, -1)

test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
test_label = np.zeros((len(test_index), 1), dtype="int64")



In [6]:
import pgl
import model
import paddle.fluid as fluid
import numpy as np
import time
from build_model import build_model

place = fluid.CPUPlace()
# 使用GPU
# place = fluid.CUDAPlace(0)

train_program = fluid.default_main_program()
startup_program = fluid.default_startup_program()
with fluid.program_guard(train_program, startup_program):
    with fluid.unique_name.guard():
        gw, loss, acc, pred = build_model(dataset,
                            config=config,
                            phase="train",
                            main_prog=train_program)

test_program = fluid.Program()
with fluid.program_guard(test_program, startup_program):
    with fluid.unique_name.guard():
        _gw, v_loss, v_acc, v_pred = build_model(dataset,
            config=config,
            phase="test",
            main_prog=test_program)


test_program = test_program.clone(for_test=True)

exe = fluid.Executor(place)


In [7]:
epoch = 400
exe.run(startup_program)

# 将图数据变成 feed_dict 用于传入Paddle Excecutor
feed_dict = gw.to_feed(dataset.graph)

for epoch in range(epoch):
    # Full Batch 训练
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(train_index, dtype="int64")
    feed_dict["node_label"] = np.array(train_label, dtype="int64")
    
    train_loss, train_acc = exe.run(train_program,
                                feed=feed_dict,
                                fetch_list=[loss, acc],
                                return_numpy=True)

    # Full Batch 验证
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    print("Epoch", epoch, "Train Acc", train_acc[0], "Valid Acc", val_acc[0])

Epoch 0 Train Acc 0.011586104 Valid Acc 0.21307041
Epoch 1 Train Acc 0.1619207 Valid Acc 0.14864384
Epoch 2 Train Acc 0.16822097 Valid Acc 0.14850146
Epoch 3 Train Acc 0.15115327 Valid Acc 0.14864384
Epoch 4 Train Acc 0.17687051 Valid Acc 0.30718303
Epoch 5 Train Acc 0.23973091 Valid Acc 0.19705275
Epoch 6 Train Acc 0.21251869 Valid Acc 0.15191856
Epoch 7 Train Acc 0.17293729 Valid Acc 0.24788211
Epoch 8 Train Acc 0.24222253 Valid Acc 0.33010608
Epoch 9 Train Acc 0.29355022 Valid Acc 0.32761443
Epoch 10 Train Acc 0.31663343 Valid Acc 0.33010608
Epoch 11 Train Acc 0.32608387 Valid Acc 0.33031964
Epoch 12 Train Acc 0.33030185 Valid Acc 0.3422795
Epoch 13 Train Acc 0.34192353 Valid Acc 0.3565886
Epoch 14 Train Acc 0.35288674 Valid Acc 0.37132484
Epoch 15 Train Acc 0.37276644 Valid Acc 0.38314232
Epoch 16 Train Acc 0.3833559 Valid Acc 0.387912
Epoch 17 Train Acc 0.38841033 Valid Acc 0.423578
Epoch 18 Train Acc 0.40932226 Valid Acc 0.43340215
Epoch 19 Train Acc 0.42106855 Valid Acc 0.433829

Epoch 164 Train Acc 0.6417918 Valid Acc 0.67380935
Epoch 165 Train Acc 0.6414359 Valid Acc 0.6753755
Epoch 166 Train Acc 0.64097315 Valid Acc 0.6758739
Epoch 167 Train Acc 0.6411867 Valid Acc 0.67089057
Epoch 168 Train Acc 0.64172065 Valid Acc 0.6738805
Epoch 169 Train Acc 0.64180964 Valid Acc 0.6755179
Epoch 170 Train Acc 0.64367837 Valid Acc 0.6696092
Epoch 171 Train Acc 0.6414537 Valid Acc 0.6757315
Epoch 172 Train Acc 0.64246815 Valid Acc 0.67288387
Epoch 173 Train Acc 0.6393358 Valid Acc 0.6585036
Epoch 174 Train Acc 0.6291023 Valid Acc 0.6707482
Epoch 175 Train Acc 0.63759166 Valid Acc 0.670677
Epoch 176 Train Acc 0.63842815 Valid Acc 0.66355807
Epoch 177 Train Acc 0.6372713 Valid Acc 0.6758739
Epoch 178 Train Acc 0.64444363 Valid Acc 0.66782945
Epoch 179 Train Acc 0.63689756 Valid Acc 0.67651457
Epoch 180 Train Acc 0.6444793 Valid Acc 0.663914
Epoch 181 Train Acc 0.63419235 Valid Acc 0.6753755
Epoch 182 Train Acc 0.6440343 Valid Acc 0.67501956
Epoch 183 Train Acc 0.6452267 Valid

Epoch 326 Train Acc 0.6606037 Valid Acc 0.69089484
Epoch 327 Train Acc 0.6633089 Valid Acc 0.69708836
Epoch 328 Train Acc 0.6646971 Valid Acc 0.69288814
Epoch 329 Train Acc 0.6628284 Valid Acc 0.6938136
Epoch 330 Train Acc 0.6633801 Valid Acc 0.69865453
Epoch 331 Train Acc 0.66451913 Valid Acc 0.69360006
Epoch 332 Train Acc 0.6625614 Valid Acc 0.6933153
Epoch 333 Train Acc 0.6644123 Valid Acc 0.6953086
Epoch 334 Train Acc 0.6638428 Valid Acc 0.6932441
Epoch 335 Train Acc 0.6614758 Valid Acc 0.6958069
Epoch 336 Train Acc 0.6631843 Valid Acc 0.6979426
Epoch 337 Train Acc 0.66499966 Valid Acc 0.69573575
Epoch 338 Train Acc 0.6666014 Valid Acc 0.6936712
Epoch 339 Train Acc 0.6631843 Valid Acc 0.6973731
Epoch 340 Train Acc 0.66549796 Valid Acc 0.6973731
Epoch 341 Train Acc 0.66613865 Valid Acc 0.6975155
Epoch 342 Train Acc 0.6630953 Valid Acc 0.69651884
Epoch 343 Train Acc 0.66564035 Valid Acc 0.6967324
Epoch 344 Train Acc 0.66656584 Valid Acc 0.6996512
Epoch 345 Train Acc 0.6679362 Valid A

In [8]:
feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]

In [10]:
submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission.csv", index=False)